In [2]:
#IMPORTS
import os
import sys
import tensorflow as tf
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pickle
from dataShaping import *
from savePerf import *
import scipy.io.wavfile
import time
from tensorflow.python.tools import freeze_graph
from tensorflow.keras import layers
import tensorflow.keras as kr
from keras.callbacks import EarlyStopping, ModelCheckpoint
import IPython


Using TensorFlow backend.


In [3]:
#Create directory of experiment and tf_logs/run-X
modelName = "Code_v1"
# create directory experiment
date = time.strftime("%Y-%m-%d-%H-%M")
path = os.path.join("experiments",date)
if not os.path.isdir(path):
    os.makedirs(path)
    #experiment/"date"/temp will contain the backuped model parameters
    pathTemp = os.path.join(path,'temp')
    os.makedirs(pathTemp)
    # if you run the file two time in a minute
else :
    date = date+'(2)'
    path = os.path.join("experiments",date)
    os.makedirs(path)
    pathTemp = os.path.join(path,'temp')
    os.makedirs(pathTemp)
	
# directory that will contain tensorboard information
pathLog = 'tf_logs'
if not os.path.isdir(pathLog):
    os.makedirs(pathLog)
pathLog = "{}/run-{}/".format(pathLog,date)

version = tf.__version__
print ("version {} of tensorflow".format(version))

version 2.0.0 of tensorflow


In [4]:
#############################
# Model parameters
#############################
trainTestRatio = 0.85
#if you cannot load all the data set in Ram specify wich part you want to load (0 means all the dataset)
maxSize = int(np.floor(44100*8.7))#11.4
num_step = 32                                         #time step
num_hidden = 300                                      #num of hidden units
num_class = 1                                          #size of the output
num_feature = 1                                        # size of the input
batch_size = 300    #300                                  # number of sequence taken before to compute the gradient
n_layer =1                                             #num_layer
num_epoch = 5     #100000                                 # process all the datas num_epoch times
trainDuration = 60     #60                         # or during a determined duration(second)
fileName = 'train.mat'             #dataset train/test path
fileNameValidation = 'val.mat' # dataset validation path

In [4]:
#############################
# Loading data
#############################
matrix = sio.loadmat(fileName)
matrix = matrix['train']
if maxSize ==0:
    maxSize = len(matrix)
    print(maxSize)
# to do shuffle matrix by num_step length
train_input,train_output,test_input,test_output = splitShuffleData(matrix,num_step,trainTestRatio,maxSize,1)
###############################
#   validation dataset and emulate guitar signal
###############################
matrixVal = sio.loadmat(fileNameValidation)
matrixVal = matrixVal['val']
val_input,val_output = shapeData(matrixVal,num_step,maxSize,1)
val_input_listen,val_output_not = shapeData(matrixVal,1,maxSize,1)
print("shape IN train {}".format(np.shape(train_input)))
print("shape OUT train {}".format(np.shape(train_output)))
print("shape IN test {}".format(np.shape(test_input)))
print("shape OUT test {}".format(np.shape(test_output)))
print("shape IN val {}".format(np.shape(val_input)))
print("shape OUT val {}".format(np.shape(val_output)))
print ("Data loaded")

shape IN train (326118, 32, 1)
shape OUT train (326118, 1, 1)
shape IN test (57519, 32, 1)
shape OUT test (57519, 1, 1)
shape IN val (383637, 32, 1)
shape OUT val (383637, 1, 1)
Data loaded


In [5]:
model=0
model = tf.keras.Sequential()
model.add(kr.layers.LSTM(32,input_shape=(num_step,num_feature),return_sequences=False))
model.add(kr.layers.Dense(32, activation='relu'))
model.add(kr.layers.Dense(1, activation='relu'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                4352      
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 5,441
Trainable params: 5,441
Non-trainable params: 0
_________________________________________________________________


In [6]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, verbose=0)
]

model.compile(loss='mse', optimizer=kr.optimizers.SGD(lr=0.05), metrics=['acc'])
model.fit(train_input,train_output, epochs=num_epoch, callbacks=callbacks)
model.evaluate(test_input, test_output)
model.summary()

Train on 326118 samples
Epoch 1/5
326118/326118 [==============================] - 102s 313us/sample - loss: 0.0247 - acc: 6.1327e-05
Epoch 2/5
   512/326118 [..............................] - ETA: 1:46 - loss: 0.0248 - acc: 0.0000e+00

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


326118/326118 [==============================] - 104s 320us/sample - loss: 0.0247 - acc: 6.1327e-05
Epoch 3/5
326118/326118 [==============================] - 101s 309us/sample - loss: 0.0247 - acc: 6.1327e-05
Epoch 4/5
326118/326118 [==============================] - 94s 287us/sample - loss: 0.0247 - acc: 6.1327e-05
Epoch 5/5
326118/326118 [==============================] - 91s 279us/sample - loss: 0.0247 - acc: 6.1327e-05
57519/1 [===================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                4352      
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 5,441
Trainable params: 5,441
Non-trainable params: 0
_________________________________________________________________


In [7]:
predictions = model.predict(np.array(val_input))

In [15]:
IPython.display.Audio(val_input_listen, rate=44100)

toPrint=predictions
import math
%matplotlib notebook
x=np.arange(0,150000,1)
fig=plt.figure()
ax=fig.add_axes([0,0, 1 , 1 ])

val=val_output_not.reshape(len(val_output_not),1)
ax.plot(toPrint[1000:15000])
ax.plot(val[1000:15000])


print(predictions.shape)
print(val_output_not.shape)
print(val_input.shape)

plt.rcParams['figure.figsize'] =[20,5]
#despues, cada plot:
plt.plot(matrixVal[:,1])

scipy.io.wavfile.write(os.path.join(path,'target.wav'),44100,val_output)
scipy.io.wavfile.write(os.path.join(path,'prediction.wav'),44100,predictions)
scipy.io.wavfile.write(os.path.join(path,'input.wav'),44100,val_input_listen)

ValueError: Array audio input must be a 1D or 2D array

In [9]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model.h5")
print("Saved model to disk")


Saved model to disk
